In [1]:
import os
import sys
import datetime
from argparse import ArgumentParser
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [2]:
DATASET_PATH = '/USER/kaggle/building/data/'

In [3]:
data = pd.read_csv(os.path.join(DATASET_PATH, 'train', 'train.csv'))
data.head()

,ID,Target
0,06004A01002010000,10_콘크리트외벽
1,61673A01001310000,10_콘크리트외벽
2,54554A01011750001,20_조적외벽
3,50653A09001530000,10_콘크리트외벽
4,01743001003600009,10_콘크리트외벽


In [4]:
data.loc[(data.Target == '50_기타외벽')]

,ID,Target
37,63034901002670000,50_기타외벽
606,47523002007170006,50_기타외벽
684,54078A0620012100000,50_기타외벽
1388,17794A04000000000,50_기타외벽
1821,29160901006020000,50_기타외벽
1822,29160901006850010,50_기타외벽
1823,10863905000350036,50_기타외벽
1824,28378901016970000,50_기타외벽
1825,34068001006390008,50_기타외벽
2254,28132A05001010030,50_기타외벽


In [5]:
from torch.utils.data import Dataset
import cv2

class CustomDataset(Dataset):

    def __init__(self, root, mode):
        super(CustomDataset, self).__init__()
        if mode == 'val':
            self.mode = 'train'
        else:
            self.mode = mode
        self.data_dir = os.path.join(root, self.mode, 'images')

        if mode == 'train':
            self.data = pd.read_csv(os.path.join(root, self.mode, f'{self.mode}.csv'))
            self.data = self.data[:int(TRAIN_RATIO*len(self.data))]
        elif mode == 'val':
            self.data = pd.read_csv(os.path.join(root, self.mode, f'{self.mode}.csv'))
            self.data = self.data[int(TRAIN_RATIO*len(self.data)):].reset_index()
        else:
            self.data = pd.read_csv(os.path.join(root, 'sample_submission.csv'))
        
        # Target 카테고리 -> 정수
        self.data = self.data.replace(['10_콘크리트외벽', '20_조적외벽', '30_판넬외벽', '40_유리외벽', '50_기타외벽'], [0, 1, 2, 3, 4])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        b_id = self.data['ID'][idx]

        try:
            img = cv2.imread(os.path.join(self.data_dir, b_id + '.png'))
            img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_LINEAR)
            img = cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
        except:
            img = np.zeros((256, 256, 3))

        img = torch.tensor(img, dtype=torch.float32)
        img = img.permute(2, 0, 1)

        if self.mode == 'test':
            target = []
        else:
            target = self.data['Target'][idx]
            target = torch.tensor(target, dtype=torch.long)

        return (b_id+'.png', img, target)

In [6]:
from sklearn.metrics import f1_score

def custom_evaluate(label, prediction):
    label = list(map(int, label))
    prediction = list(map(int, prediction))
    return f1_score(label, prediction, average='macro')

In [7]:
# !pip install efficientnet-pytorch

## EfficientNet-PyTorch
[github](https://github.com/lukemelas/EfficientNet-PyTorch)

![](https://raw.githubusercontent.com/tensorflow/tpu/master/models/official/efficientnet/g3doc/params.png)

In [8]:
from efficientnet_pytorch import EfficientNet

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.channel = 1792 #b1: 1280, b4: 1792
        self.h = 8
        self.w = 8

        # 1 epoch train time aprox. 25 min
        self.en = EfficientNet.from_pretrained('efficientnet-b4').to(device)
        self.avp = nn.AvgPool2d(self.h, stride=1, padding=0)
        self.fc_layer = nn.Sequential(
            nn.Linear(in_features=self.channel, out_features=5, bias=True),
            nn.Softmax(dim=1)
        ).to(device)

    def forward(self, x):
        features = self.en.extract_features(x)
        x = self.avp(features)
        x = x.squeeze()
        x = self.fc_layer(x)
        return x

In [9]:
def train():
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    else:
        torch.manual_seed(42)

    model = CNN().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

    # DataLoader
    training_params = {'batch_size': batch_size,
                    'shuffle': True,
                    'drop_last': True,
                    'collate_fn': None,
                    'num_workers': 0}
    validating_params = {'batch_size': batch_size,
                    'shuffle': True,
                    'drop_last': True,
                    'collate_fn': None,
                    'num_workers': 0}

    training_set = CustomDataset(root=DATASET_PATH, mode='train')
    training_generator = DataLoader(training_set, **training_params)
    validating_set = CustomDataset(root=DATASET_PATH, mode='val')
    validating_generator = DataLoader(validating_set, **validating_params)

    # Print trainable number of parameters
    print("------------------------------------------------------------")
    total_params = sum(p.numel() for p in model.parameters())
    print("num of parameter : ", total_params)
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("num of trainable parameter :", trainable_params)
    print("------------------------------------------------------------")

    # Train Model
    val_acc_history = []
    for epoch in range(num_epochs):
        epoch_start = datetime.datetime.now()

        running_loss = 0.0
        running_score = 0.0
        for iter, (_, imgs, target) in enumerate(training_generator):
            try:
                imgs, target = imgs.to(device), target.to(device)

                optimizer.zero_grad()
                output = model(imgs)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                output = torch.argmax(output, dim=1)
                running_score += custom_evaluate(target, output)

                if iter % 10 == 9:
                    print(f'Train Epoch: {epoch} [ {iter * len(imgs)}/{len(training_generator.dataset)} ({100. * iter / len(training_generator)}%)]\tLoss: {loss.item()}')

                if iter % 20 == 19:
                    print(f'[{epoch}, {iter}] loss: {running_loss / 20}, f1 score: {running_score / 20}')
                    running_loss = 0.0
                    running_score = 0.0

            except Exception as e:
                print('Error:', e)
                continue

        lr_scheduler.step()

        print(f'\nEpoch: {epoch}, loss: {loss.item()}')
        print(f'(epoch)time: {datetime.datetime.now() - epoch_start}')

        if epoch % save_epochs == 0:
            torch.save(model.state_dict(), os.path.join(log_dir,f'epoch_{epoch}.pth'))
            print('Model Saved')

        if epoch % val_epochs == 0:
            val_start = datetime.datetime.now()
            model.eval()
            with torch.no_grad():
                val_targets = []
                val_outputs = []
                for iter, (_, imgs, target) in enumerate(validating_generator):
                    try:
                        imgs = imgs.to(device)
                        target = target.to(device)

                        output = model(imgs)
                        output = torch.argmax(output, dim=1)

                        val_targets.append(target)
                        val_outputs.append(output)

                        if iter % 500 == 0:
                            print(f'epoch: {epoch}, batch val: {custom_evaluate(target, output)}')

                    except Exception as e:
                        print(f'Validation Error: {e}')
                        continue

            val_targets = torch.cat(val_targets, dim=0)
            val_outputs = torch.cat(val_outputs, dim=0)
            val_score = custom_evaluate(val_targets, val_outputs)
            print(f'(val)time: {datetime.datetime.now() - val_start}')
            print(f'Validation Score: {val_score}\n===============\n\n')
            val_acc_history.append([epoch, val_score])

            model.train()

    # Print Validation Scores
    for epoch, score in val_acc_history:
        print(f'Epoch: {epoch}, Score: {score}')

    # save final result
    torch.save(model.state_dict(), os.path.join(log_dir,'/last.pth'))
    print('\n\nModel Saved, Finishing training')


In [10]:
def test():

    # weight = weight
    batch_size = 128
    prediction = prediction_file

    # Load Model
    model = CNN().to(device)
    model.load_state_dict(torch.load(os.path.join(log_dir,f'{weight}')))
    model.requires_grad_(False)
    model.eval()

    # DataLoader
    testing_params = {'batch_size': batch_size,
                    'shuffle': False,
                    'drop_last': False,
                    'collate_fn': None,
                    'num_workers': 0}

    testing_set = CustomDataset(root=DATASET_PATH, mode='test')
    testing_generator = DataLoader(testing_set, **testing_params)
    print('test dataset ready---')

    # Test
    test_bids = []
    test_preds = []
    for iter, (b_ids, imgs, _) in enumerate(testing_generator):
        with torch.no_grad():
            try:
                imgs = imgs.to(device)
                output = model(imgs)
                output = torch.argmax(output, dim=1)
                test_bids.extend(b_ids)
                test_preds.append(output)

                if iter % 100 == 0:
                    print(f'iter: {iter}')

            except Exception as e:
                print('Testing Error:', e)
                sys.exit()

    test_preds = torch.cat(test_preds, dim=0)
    test_preds = test_preds.cpu().data.numpy()
    save_result(test_bids, test_preds, path=prediction)


In [11]:
def save_result(names, y_pred, path='prediction2.csv'):
    try:
        int2code = {0: '10_콘크리트외벽', 
                    1: '20_조적외벽',
                    2: '30_판넬외벽',
                    3: '40_유리외벽',
                    4: '50_기타외벽'}
        
        y_pred = list(map(lambda x : int2code[x], y_pred))
        with open(os.path.join(log_dir,path), 'w', encoding='utf-8', newline='') as f:
            f.write('ID,Target\n')
            for i, pred in enumerate(y_pred):
                filename = names[i].split('.')[0]
                f.write(f'{str(filename)},{str(pred)}\n')
        print(f'Saved result {path}')

    except Exception as e:
        print(f'Fail to save {e}')

In [12]:
# for `libpng error: Read Error`
# !pip install update libpng-bins

In [13]:
num_epochs = 5
learning_rate = 2e-4
batch_size = 32
val_epochs = 1
save_epochs = 1
weight = 'last.pth'
prediction_file = 'prediction.csv'

In [14]:
import datetime

os.makedirs('./log/', exist_ok=True)
nth_trial = len(os.listdir('./log'))-1
log_dir = (f'./log/trial{nth_trial}')
os.makedirs(log_dir)

DATASET_PATH = '/USER/kaggle/building/data/'
TRAIN_RATIO = 0.9

start = datetime.datetime.now()

# Set device (cpu or gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

train()
print(f'TRAIN ended---duration:{datetime.datetime.now() - start}')

cuda


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth


  0%|          | 0.00/74.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b4
------------------------------------------------------------
num of parameter :  19350581
num of trainable parameter : 19350581
------------------------------------------------------------
Train Epoch: 0 [ 288/16586 (1.7374517374517375%)]	Loss: 1.5137020349502563
Train Epoch: 0 [ 608/16586 (3.667953667953668%)]	Loss: 1.374488353729248
[0, 19] loss: 1.48594172000885, f1 score: 0.36863464653406786
Train Epoch: 0 [ 928/16586 (5.598455598455598%)]	Loss: 1.1622593402862549
Train Epoch: 0 [ 1248/16586 (7.528957528957529%)]	Loss: 1.1642582416534424
[0, 39] loss: 1.216974174976349, f1 score: 0.5245664150880626
Train Epoch: 0 [ 1568/16586 (9.45945945945946%)]	Loss: 1.1036995649337769
Train Epoch: 0 [ 1888/16586 (11.38996138996139%)]	Loss: 1.214789867401123
[0, 59] loss: 1.1464894950389861, f1 score: 0.533198850901746
Train Epoch: 0 [ 2208/16586 (13.32046332046332%)]	Loss: 1.1277358531951904
Train Epoch: 0 [ 2528/16586 (15.250965250965251%)]	Loss: 1.

libpng error: Read Error


...

(val)time: 0:05:35.386633
Validation Score: 0.5339131493658554


Train Epoch: 4 [ 288/16586 (1.7374517374517375%)]	Loss: 1.0693222284317017
Train Epoch: 4 [ 608/16586 (3.667953667953668%)]	Loss: 1.0517860651016235
[4, 19] loss: 1.0493452727794648, f1 score: 0.5971817459512112
Train Epoch: 4 [ 928/16586 (5.598455598455598%)]	Loss: 1.1438467502593994
Train Epoch: 4 [ 1248/16586 (7.528957528957529%)]	Loss: 0.9788084626197815
[4, 39] loss: 1.0538599461317062, f1 score: 0.5530541057985339
Train Epoch: 4 [ 1568/16586 (9.45945945945946%)]	Loss: 1.06111478805542
Train Epoch: 4 [ 1888/16586 (11.38996138996139%)]	Loss: 1.0646768808364868
[4, 59] loss: 1.04953553378582, f1 score: 0.5999684152611388
Train Epoch: 4 [ 2208/16586 (13.32046332046332%)]	Loss: 1.03908109664917
Train Epoch: 4 [ 2528/16586 (15.250965250965251%)]	Loss: 1.095600962638855
[4, 79] loss: 1.0736371845006942, f1 score: 0.5651593298583318
Train Epoch: 4 [ 2848/16586 (17.18146718146718%)]	Loss: 1.0010617971420288
Train Epoch: 4 [ 

libpng error: Read Error


Train Epoch: 4 [ 5408/16586 (32.625482625482626%)]	Loss: 1.0951530933380127
Train Epoch: 4 [ 5728/16586 (34.55598455598456%)]	Loss: 1.0318260192871094
[4, 179] loss: 1.0547442764043808, f1 score: 0.5629687400676386
Train Epoch: 4 [ 6048/16586 (36.486486486486484%)]	Loss: 1.0547056198120117
Train Epoch: 4 [ 6368/16586 (38.41698841698842%)]	Loss: 1.1239038705825806
[4, 199] loss: 1.0664322465658187, f1 score: 0.5584836419208716
Train Epoch: 4 [ 6688/16586 (40.34749034749035%)]	Loss: 1.0572574138641357
Train Epoch: 4 [ 7008/16586 (42.277992277992276%)]	Loss: 1.0321934223175049
[4, 219] loss: 1.0351507723331452, f1 score: 0.5789414365310785
Train Epoch: 4 [ 7328/16586 (44.20849420849421%)]	Loss: 1.1515254974365234
Train Epoch: 4 [ 7648/16586 (46.13899613899614%)]	Loss: 1.024595856666565
[4, 239] loss: 1.041838574409485, f1 score: 0.6098482126716325
Train Epoch: 4 [ 7968/16586 (48.06949806949807%)]	Loss: 1.0299476385116577
Train Epoch: 4 [ 8288/16586 (50.0%)]	Loss: 1.1374868154525757
[4, 25

libpng error: Read Error


(val)time: 0:05:29.485881
Validation Score: 0.5275974077765792


Epoch: 0, Score: 0.5135233857136235
Epoch: 1, Score: 0.5390671584645487
Epoch: 2, Score: 0.5363153567318709
Epoch: 3, Score: 0.5339131493658554
Epoch: 4, Score: 0.5275974077765792


Model Saved, Finishing training
TRAIN ended---duration:5:13:04.955022


In [15]:
import datetime

start = datetime.datetime.now()
print(start)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# log_dir = './log/trial1'
# weight = 'epoch_2.pth'

# test()
# print(f'TEST ended---duration:{datetime.datetime.now() - start}')

2021-11-12 06:50:15.605501
